<a href="https://colab.research.google.com/github/LuciaCorreaDaSilva/python-google-speech-scripts/blob/master/TesteCodigoBase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive

import sys, io, os, shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from pyaudioclassification.limpar import limpar

from pyaudioclassification import *

    
dirName_data = list()
dirName_results = list()
my_dirs = [x[0] for x in os.walk("//data//usar no modelo")]
for i in my_dirs:
    if (os.path.basename(os.path.normpath(i)) == "dripping"):
        dirName_data.append(os.path.split(i)[0])
        dirName_results.append(os.path.split(i)[0].replace("data", "resultados"))

posssiveis_redes = ["cnn2d", "cnn", "lstm", "nn"]

n = len(dirName_data)
local_dados = ''
parent_dir = ''

for i in range(0, n):
    local_dados = dirName_data[i]
    for type in posssiveis_redes:
        parent_dir = dirName_results[i] + '//' + type
        os.chdir(parent_dir)
        
        # if (type != "cnn2d"):
        #     file = os.listdir(dirName_results[i] + '//' + "cnn2d")
        #     for f in file:
        #         if (os.path.splitext(f)[1] == ".png"):
        #             continue
        #         else:
        #             shutil.copy(dirName_results[i] + '//' + "cnn2d" + "//" + f, parent_dir)   
        #     limpar(limpar=False, feat_label=False, model_history=True, samples=True)

        # type = "cnn2d"
        # local_dados = "D://PythonVSCode//sintetico//data//usar no modelo//sinais concatenado SNR//com operacao estacionaria//SNR menos 12 db dripping menos 24 db nock"
        # parent_dir = "D://PythonVSCode//sintetico//resultados//usar no modelo//sinais concatenado SNR//com operacao estacionaria//SNR menos 12 db dripping menos 24 db nock//cnn2d"
        # os.chdir(parent_dir)

        random_state = 42
        rand = 0
        epochs = 100
        amostras_separar = 'train_test_val'

        # step 0: clean directory
        #limpar(limpar=False, feat_label=False, model_history=False, samples=False)
            
        # step 1: preprocessing
        if np.DataSource().exists("./feat.npy") and np.DataSource().exists("./label.npy"):
            features, labels = np.load('./feat.npy'), np.load('./label.npy')
        else:
            features, labels = feature_extraction(local_dados)
            np.save('./feat.npy', features)
            np.save('./label.npy', labels)
            
        if type == posssiveis_redes[0]:
            for j in range(1,4):
                test = dirName_results[i] + '//' + posssiveis_redes[j]
                np.save(test + '//feat.npy', features)
                np.save(test + '//label.npy', labels)

        # If you don't specify the random_state in your code, then every time you run(execute) your code a new random value is generated and the train and test datasets would have different values each time.
        # However, if a fixed value is assigned like random_state = 42 then no matter how many times you execute your code the result would be the same .i.e, same values in train and test datasets.
        # https://stackoverflow.com/questions/28064634/random-state-pseudo-random-number-in-scikit-learn

        # step 2: training       
        if np.DataSource().exists("./model.h5"):
            from keras.models import load_model
            model = load_model('./model.h5')
            history = np.load('./history.npy', allow_pickle=True).item()
        else:
            model, history = train(features, labels, type, random_state=random_state, epochs=epochs)
            model.save('./model.h5')
            np.save('./history.npy', history.history)

        # step 3: prediction, accuracy and loss
        X_train = np.load("X_train.npy")
        y_train = np.load("y_train.npy")
        X_test = np.load("X_test.npy")
        y_test = np.load("y_test.npy")

        filename = 'Predict and accuracy - ' + type + ' ' + amostras_separar
        sys.stdout = open(filename+".txt", "w")
        print(filename)
        print('')
        

        pred = predict(model, './X_test.npy', type=type)

        _, _, train_acc, test_acc = acc_loss_fig(
            X_train, y_train, model, history, type, amostras_separar, epochs,
            X_test=X_test, y_test=y_test, save_fig=True, close_fig=True)

        print('')
        print_leaderboard(pred, local_dados, rand)
        sys.stdout.close()


        # step 4: Confusion matrix
        labels = list(filter(lambda x: os.path.isdir(
            os.path.join(local_dados, x)), os.listdir(local_dados)))
        cm_analysis(y_test, pred, labels, type, amostras_separar, figsize=(10, 10),
                    save_fig=True, close_fig=True)

        # step 5: Classification report
        plot_classification_report(y_test, pred, labels, type, amostras_separar, porcentagem=True,
                                figsize=(10, 10), ax=None, save_fig=True, close_fig=True)

        # # step 6: obtain summary
        # summary_string = obter_model_summary(model, type, amostras_separar)


        # step 4: Confusion matrix
        labels = list(filter(lambda x: os.path.isdir(
            os.path.join(local_dados, x)), os.listdir(local_dados)))
        cm_analysis(y_test, pred, labels, type, amostras_separar, figsize=(10, 10),
                    save_fig=True, close_fig=True)

        # step 5: Classification report
        plot_classification_report(y_test, pred, labels, type, amostras_separar, porcentagem=False,
                                figsize=(10, 10), ax=None, save_fig=True, close_fig=True)

        # # step 6: obtain summary
        # summary_string = obter_model_summary(model, type, amostras_separar)
        
        # from numba import cuda 
        # device = cuda.get_current_device()
        # device.reset()

ModuleNotFoundError: ignored

# Nova seção

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# New Section